In [1]:
from astropy.io import fits
from astropy.table import Table, join, unique

from astropy.coordinates import SkyCoord
import astropy.units as u

import pyphot

import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt


from multiprocessing import set_start_method
set_start_method('spawn')

plt.style.use('stefan.mplstyle')

In [2]:
# Load in SDSS-V access tokens and the Gaia crossmatch table
with open('creds.txt') as f:
    creds = f.read().splitlines()
    
catalog = Table.read('data/SDSS_V_Gaia_xmatch.csv')

In [3]:
lib = pyphot.get_library()
wise_photo = ['WISE_RSR_W1', 'WISE_RSR_W2', 'WISE_RSR_W3', 'WISE_RSR_W4']
wise_zeros = [lib[filt].Vega_zero_mag for filt in wise_photo]
wise_pivots = [lib[filt].lpivot.to('angstrom').value for filt in wise_photo]

gaia_photo = ['Gaia_G', 'Gaia_BP', 'Gaia_RP']
gaia_zeros = [lib[filt].Vega_zero_mag for filt in gaia_photo]
gaia_pivots = [lib[filt].lpivot.to('angstrom').value for filt in gaia_photo]

wavls = np.array(gaia_pivots + wise_pivots)

In [4]:
import requests
import os

with open('creds.txt') as f:
    creds = f.read().splitlines()

session = requests.Session()
session.auth = (creds[0],creds[1])

def download_spectrum(row, outfile = 'tempfiles/'):    
    # use the requests package to download the fits file of a given row quickly
    plate = row['spec_file'].split('-')[1]
    mjd = row['spec_file'].split('-')[2]
    file = row['spec_file']
    
    filepath = 'https://data.sdss5.org/sas/sdsswork/bhm/boss/spectro/redux/v6_1_0/spectra/full/{}/{}/{}'.format(plate, mjd, file)
    
    with open('tempfiles/{}'.format(row['spec_file']), 'wb') as f:
        f.write(session.get(filepath).content)
        
    file = fits.open('tempfiles/{}'.format(row['spec_file']))
                
    return file

def clear_directory(path = 'tempfiles/'):
    os.system('rm {}*'.format(path))

def download_sed(i):
    coord = SkyCoord(catalog['ra'][i], catalog['dec'][i], unit = u.deg)
    target = ''.join(coord.to_string('hmsdms', sep='+').split(' '))
    
    radius = 1
    sed=Table.read(f"https://vizier.cds.unistra.fr/viz-bin/sed?-c={target}&-c.rs={radius}")
    
    sed['sed_wavl'] = (299792.458/sed["sed_freq"])
    sed['sed_fl'] = (2.99792458e-05 * sed["sed_flux"] / (299792.458e4/sed["sed_freq"])**2)
    sed['sed_e_fl'] = np.sqrt(( (2.99792458e-05 / (299792.458e4/sed["sed_freq"])**2) * sed['sed_eflux'] )**2)
    
    return sed



In [5]:
def wise_mag_to_flux(mag, zeropoint, pivot):
    print(zeropoint)
    return 2.99792458e-5 / pivot**2 * (zeropoint * 10**(mag / -2.5)) 

def gaia_mag_to_flux(mag, zeropoint):
    return 10**( (float(mag) + zeropoint) / -2.5 )


mag_keys = ['phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag', 'w1mpro', 'w2mpro', 'w3mpro', 'w4mpro']

In [6]:
def blackbody(wavl, teff, radius, distance):
    wavl = wavl * 1e-6 # um input to m
    
    planck = 6.626e-34 # J / s 
    c = 2.99e8 # m / s
    kb = 1.3806e-23 #m^2 kg s^-2 K^-1
    
    si = ((2 * np.pi * planck * c**2) / (wavl**5)) * ( 1 / (np.exp( (planck * c) / (kb * teff * wavl) ) - 1)) * 1e-7 
    
    radius = radius * 6.957e8
    distance = distance * 3.086775e16
        
    return si * (radius / distance)**2
    
def double_blackbody(wavls, teff_1, radius_1, teff_2, radius_2, distance):
    return blackbody(wavls, teff_1, radius_1, distance) + blackbody(wavls, teff_2, radius_2, distance)

def single_cost_function(params, wavls, obs_flux, e_obs_flux = None):
    teff, radius, distance = params['teff'], params['radius'], params['distance']
    
    model_flux = blackbody(wavls, teff, radius, distance)
    
    if e_obs_flux is not None:
        return (model_flux - obs_flux)**2 / (e_obs_flux)**2
    else:
        return (model_flux - obs_flux)**2
    
def double_cost_function(params, wavls, obs_flux, e_obs_flux = None):
    teff_1, radius_1, teff_2, radius_2, distance = params['teff'], params['radius'],  params['disk_teff'], params['disk_radius'], params['distance']
    
    model_flux = double_blackbody(wavls, teff_1, radius_1, teff_2, radius_2, distance)
    
    if e_obs_flux is not None:
        return (model_flux - obs_flux)**2 / (e_obs_flux)**2
    else:
        return (model_flux - obs_flux)**2

In [7]:
import lmfit

def fit_functions(i):
    sed = download_sed(i)
    
    dist = 100 / float(catalog['parallax'][i])
    
    params = lmfit.Parameters()
    
    params.add('teff', value = 5000, min = 0, max = 100000, vary = True)
    params.add('radius', value = 0.001, min = 0.000001, max = 0.05, vary = True)
    params.add('distance', value = dist, min = 1, max = 2000, vary = False)
    
    result_single = lmfit.minimize(single_cost_function, params, kws = dict(wavls = sed['sed_wavl'], obs_flux = sed['sed_fl'], e_obs_flux = sed['sed_e_fl']), method = 'nedler')
    
    
    params = lmfit.Parameters()
    
    params.add('teff', value = 5000, min = 0, max = 100000, vary = True)
    params.add('radius', value = 0.001, min = 0.000001, max = 1e10, vary = True)
    params.add('disk_teff', value = 5000, min = 0, max = 100000, vary = True)
    params.add('disk_radius', value = 0.001, min =  0.000001, max = 1e10, vary = True)
    params.add('distance', value = dist, min = 1, max = 2000, vary = False)
    
    result_double = lmfit.minimize(double_cost_function, params, kws = dict(wavls = sed['sed_wavl'], obs_flux = sed['sed_fl'], e_obs_flux = sed['sed_e_fl']), method = 'nedler')
    
    print('{} was processed'.format(i))
    
    return result_single, result_double

def plot_functions(i, result_single, result_double):
    sed = download_sed(i)
    #dat = download_spectrum(catalog[i])
    #clear_directory()
    
    fig = plt.figure()

    font = 20
    
    ax = fig.add_subplot()
    ax.set_xlabel('Wavelength [$\mu m$]')
    ax.set_ylabel('Flux $[erg/cm^2/s/\AA]$')
      
    ax.errorbar(sed['sed_wavl'], sed['sed_fl'], yerr = sed['sed_e_fl'], fmt = 'o', c = 'k', label = 'Photometry')
    #ax.plot(1e-4*10**dat[1].data['loglam'], (1e-17*dat[1].data['flux']) , zorder = 0, c = 'k', alpha = 0.3, label = 'SDSS-V Spectroscopy')
    
    wavls = np.linspace(0, 5, 1000)
    irr_single = blackbody(wavls, result_single.params['teff'].value, result_single.params['radius'].value, result_single.params['distance'].value)
    plt.plot(wavls, irr_single, label = 'Single Blackbody')
    
    
    irr_double = double_blackbody(wavls, result_double.params['teff'].value, result_double.params['radius'].value, 
                             result_double.params['disk_teff'].value, result_double.params['disk_radius'].value, result_double.params['distance'].value)
    plt.plot(wavls, irr_double, label = 'Double Blackbody')
    
    plt.ylim(0, 2e-15)
    
    ymin, ymax = plt.ylim()
    xmin, xmax = plt.xlim()
    
    plt.text(0.05*(xmax - xmin) + xmin, 0.93*(ymax - ymin) + ymin, '$\chi^2_r$ ratio {:2.2f}'.format(result_single.redchi / result_double.redchi), fontsize = 18)
    
    plt.legend(framealpha=0)
    plt.close()
        
    return fig

In [8]:
import multiprocessing
from multiprocessing import Process, set_start_method
import time


if __name__ == "__main__": 
    start = time.perf_counter()
    
    processes = [multiprocessing.Process(target=fit_functions, args=[i]) for i in range(len(catalog[0:10]))]
    
    for process in processes:
        process.start()
        
    for process in processes:
        process.join()
        
    finish = time.perf_counter()

    print(f'It took {finish-start:.2f} second(s) to finish')
     
    #result = fit_functions(0)
    print('Done!')

It took 0.30 second(s) to finish
Done!
